In [140]:
#import gensim
from gensim import corpora, models, similarities
from pprint import pprint   # pretty-printer
import logging
import numpy as np
import pandas as pd
import nltk
import pymysql
import re

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# documents is a set of reviews from one doctor's Yelp page
# I just copied and pasted in 5 reviews (I think)
#documents = [u'Dr Yee is amazing. \xa0I have been seeing her for over 7 years and she delivered my first baby and has been great throughout my entire pregnancy. \xa0Highly recommend her.', u'Is it inappropriate to review your lady doc? Maybe. But I must give a shootout to my Ob/Gyn soul mate! Dr. Yee is everything that is perfect for me in a doctor. I am a high-strung, anxious individual. This and pregnancy do not mix. Dr. Yee was always so laid back and confident, it rubbed off on me! This is what I need. A confident and cool doc to keep me from thinking I am dying every time some little tiny normal thing goes wrong. She also never pressured me, I told her I didn\'t want to induce and she said ok, and instead of saying "you\'re trying not to get an epidural" she would say "you\'re not getting one". That confidence and support go a long way! She saw me on time for pretty much every appointment, and I saw her, not another doc, almost every time. Now, Dr. Yee did not deliver my baby. By this, I am still completely devastated. But from my experience, her associates are the nicest women on the planet. Going into labor on a holiday is tough, though. I don\'t blame or resent her one bit. What she did do, though, was call me right after I had my baby and talked to me for a good 15-20 minutes! Doctors don\'t do this. They just don\'t. She asked about the delivery, we laughed, it was the nicest and most caring thing I\'ve ever witnessed by a healthcare professional. She really cared! So sweet. I want to have another baby, pronto, just so I can keep seeing her! Haha, just kidding! But really, she\'s the greatest.', u"Dr. Yee has been my 5 star OBGYN for 8 years!! She is a knowledgeable doctor but over the past years the appointments have become increasingly fast-paced and rushed. When I had a miscarriage she gave me a prescription and said - it would be like a heavier period. Well, I wish. I was in intense labor-pain for 13 hrs, had an allergic reaction and therefore couldn't take any painkillers to deal with the intense pain. When I informed her at the next visit there was no sympathy nor acknowledgment that she had failed to inform me of the side effects or what this was going to be like. I stuck with her after that experience but now I think it's time to find a doc where I feel taken care of and well-informed.", u'I am a long time patient of Dr. Rebecca Yee, she has seen me through two pregnancies, and she is a perfectly competent and very capable physician. \xa0It\'s too bad HER PATIENTS NEVER GET TO SEE HER! \xa0In the last three years, she has only graced me with her presence once. \xa0Everytime I\'ve needed to be seen I was passed down to Olga, (who is very nice BTW) her Physician Assistant. \xa0Classic case of TOO MANY PATIENTS! When I first began seeing Dr. Yee six years ago, when I was pregnant with my son. I felt like she was a great doctor, who truly cared about her patients and had a personal connection with each of them. I have several friends who all see (or try to see) her. \xa0I think quite a few of her patients will agree that the personal attention and relationship has certainly changed and given way to rushed appointments in what feels like a factory churnning out patients with a quick "take two of these and call me in the morning" . \xa0Don\'t even get me started on the front dest/appointment staff. \xa0I live in Vallejo, CA and I work in Napa. \xa0I scheduled an 8:30 appointment at Dr. Yee\'s office (with none other than Olga). \xa0After getting my two kids up at 6:00am to to take them to daycare by 7 so that I can get on the road to city, I was caught in just a little rush hour morning traffic on interstate eight.y \xa0I was literally 10 like ONE ZERO minutes late and I was told that I could not been seen. \xa0Where is the compassion? \xa0Oh I forgot I was offered a 9:45 appointment and a good luck making it to your job by 10! \xa0Needless to say, I will be helping Dr. Yee out with her "too many patients problem" because now she definately has one less.', u'Dr Yee delivered our first baby and my wife has been a patient of hers throughout the pregnancy. We had a great experience with her. She is a phenomenal doctor, has great bedside manner, and patiently answered our many questions. Sometimes we had to see a nurse practitioner when Dr Yee was not available (or was off delivering babies), but we had no problem getting onto her schedule when we booked a few weeks out. She certainly lived up to her great reviews.']
#print(documents[1])

In [116]:
def call_mysql():
    # pull in data from the reviews table
    con = False
    rows = np.nan
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            sql = 'USE yelpdata;'
            cur.execute(sql)
            sql = 'SELECT id,business_id,stars,comment FROM review;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
#             for row in rows:
#                 print(row)
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [117]:
rows = call_mysql()

# UNDERSTAND THE OUTPUT OF THE MYSQL
# print(np.shape(rows))#(3616, 4)
# print(type(rows))#tuple
# print(type(rows[:][0]))# still tuple
# print(type(rows[3][3]))# int/str or whatever per the column

#print(rows[0][3])#just the review for the zero-indexed row [0]
# print(rows[1][3])#the review for the one-indexed row [1]

# confusing - not printing all rows of the first column
#print(rows[:][0])#all four columns of the data-row

documents = []#type list
ndocs = np.shape(rows)[0]

#reviews = [[review for review in rows[3]] for row in rows]
for row in rows:
    documents.append(row[3])# row[3] is type string, documents is type list

print(documents[0])# prints one review

# for doc in range(0,ndocs):
#     [review for review in rows[:][0]]# This prints the review from the id in the second []
#     print review#type str

# print(row[3])#just the review
# print(row[:])# a row of data
# print(review)# one review


Adding some more information. Another great thing about him is that he was willing to experiment with me for my fibromyalgia. I brought in a book by a specialist and he was willing to try the protocol. Many doctors would not do that. He won't do anything crazy but will consider something reasonable. He also made himself available when I was having an emergency severe toothearache. He was at an offsite seminar, but his receptionist asked him to call me and he did, prescribed something and I think he even checked in with me later on or I called back. Either way, I got the treatment I needed.For those who think it takes a while to get ahold of him or his receptionistplease keep in mind it is one doctor and one receptionist. As you can see from all of the 5 star reviews, he spends time with his patients. Be a little patient yourself and you will be soon benefit from his great treatment.


In [126]:
# Gensim code
# remove common words and tokenize
common_words = 'for a of the and to in'
stoplist = set(common_words.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:# text is a list of tokenized words from one document/review
     for token in text:# token is an individual word
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

#print texts[:2]

In [189]:
def canonicalize(word):
    #Assign all possible time values ('9:45','12:05',etc) to one word 'oclock'
    match = re.search(r'\d*'+':'+r'\d*',word)
    if match:
        #print(match.group())
        word = 'oclock'

    bads = ['dr.',"n't","'ve","'re",'wo']
    goods = ['dr','not','have','are','will']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

# This also works, better if you want to catch the error and do something, but I don't
# try: 
#     match = re.search(r'\d*'+':'+r'\d*',s)
#     test = match.group() 
# except AttributeError as e: 
#     print('error')

In [191]:
# NLTK code
# remove common words and tokenize
common_words = 'for a of the and this sf to in is if , \'s by on the to . ...  `` ? _ - / "\" 3 2 5'
# add numbers to common words
range = np.arange(0,100)
ss =''
ss = ss.join([' '+str(i) for i in range])

stoplist = set(common_words.split())

#texts: A list of lists; text: list of tokenized words from 1 doc/review
# texts = [[word for word in nltk.word_tokenize(document.lower()) if word not in stoplist]
#         for document in documents]#review in reviews 
texts = [[canonicalize(word) for word in nltk.word_tokenize(document.lower()) if word not in stoplist]
        for document in documents]#review in reviews 

print(texts[:2])

[['adding', 'some', 'more', 'information', 'another', 'great', 'thing', 'about', 'him', 'that', 'he', 'was', 'willing', 'experiment', 'with', 'me', 'my', 'fibromyalgia', 'i', 'brought', 'book', 'specialist', 'he', 'was', 'willing', 'try', 'protocol', 'many', 'doctors', 'would', 'not', 'do', 'that', 'he', 'wo', 'not', 'do', 'anything', 'crazy', 'but', 'will', 'consider', 'something', 'reasonable', 'he', 'also', 'made', 'himself', 'available', 'when', 'i', 'was', 'having', 'an', 'emergency', 'severe', 'toothearache', 'he', 'was', 'at', 'an', 'offsite', 'seminar', 'but', 'his', 'receptionist', 'asked', 'him', 'call', 'me', 'he', 'did', 'prescribed', 'something', 'i', 'think', 'he', 'even', 'checked', 'with', 'me', 'later', 'or', 'i', 'called', 'back', 'either', 'way', 'i', 'got', 'treatment', 'i', 'needed.for', 'those', 'who', 'think', 'it', 'takes', 'while', 'get', 'ahold', 'him', 'or', 'his', 'receptionistplease', 'keep', 'mind', 'it', 'one', 'doctor', 'one', 'receptionist', 'as', 'you'

In [121]:
# BAG OF WORDS
dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts
dictionary.save('/tmp/doctor.dict') # store the dictionary, for future reference
print(dictionary)
#print(type(dictionary))#<class 'gensim.corpora.dictionary.Dictionary'>
print(dictionary.token2id)

# To merge with another dictionary -  Dictionary.merge_with())# Apply bag of words to reviews & print tokenized dictionary

Dictionary(20307 unique tokens: [u'radiologists', u'yellow', u'narcotic', u'advices', u'manner.dr']...)
{u'radiologists': 15460, u'yellow': 5367, u'narcotic': 8444, u'advices': 2220, u'manner.dr': 4061, u'cancels': 16486, u'cashbring': 19033, u'hanging': 5077, u'bromelain': 17158, u'aberson': 14197, u'hastily': 12628, u'conjure': 6389, u'looking': 785, u'huggyfeelyletsdoyoga': 18750, u'month.4': 12985, u'journalmaking': 15715, u'orientationreview': 16120, u'rupture': 6832, u'appintment': 12783, u'quadruple': 6041, u'phone.after': 12513, u'superficially': 16427, u'issuescontroversies': 11899, u'lori': 9140, u'atrophying': 14002, u'everything.i': 16570, u'inwards': 17370, u'whoops': 13931, u'tweaked': 15761, u'49.dr': 14352, u'hormone': 7696, u'site.dr': 4351, u'happy.after': 10758, u'naturopathic': 5119, u'mccutie': 12842, u'deli': 17204, u'wood*': 3101, u'assumption.it': 13146, u'blackheads': 18542, u'wait.if': 6739, u'back.plus': 18552, u'reconstructions': 18092, u'surgery.this': 1811

In [7]:
# SERIALIZE & SAVE TO DISK 

# Convert tokenized documents to sparse vectors: -- [(0, 1), (4, 3),...
# new_vec = dictionary.doc2bow(documents[1].lower().split())
# print(new_vec)

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/doctor.mm', corpus) # store to disk, for later use

#print(corpus)
# looks like - [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)... (35, 2)...
# word order index -frequency of occurence?

In [8]:
# Tf-idf model
# term frequency - inverse document frequency
# higher index - occured a lot in a few docs:  log(D/d) ---  uniqueness/singularity
# D = # of doc, d = # of doc that that word appears in
# tf-idf will convert any vector from bag-of-words integer counts to Tfidf real-valued weights

# CAUTION - The same vector space (= the same set of feature ids) must be used for training as well as for subsequent vector transformations. Failure to use the same input feature space, such as applying a different string preprocessing, using different feature ids, or using bag-of-words input vectors where TfIdf vectors are expected, will result in feature mismatch during transformation calls and consequently in either garbage output and/or runtime exceptions.

tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

#  Apply tfidf to the whole corpus
corpus_tfidf = tfidf[corpus]
# for doc in corpus_tfidf:
#     print(doc)


# example transformation
# doc_bow = [(0,1),(1,3)]
# print(tfidf[doc_bow]); print('\n')

In [9]:
# Transformations can also be serialized, one on top of another, in a sort of chain:
# LSI is another transformation, which acts on the Tf-idf-weighted space

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3)
corpus_lsi = lsi[corpus_tfidf]
lsi.print_topics(3)

# Interpret my Reviews - I have only five docs (reviews) total
# The first five reviews all relate moderately to the first topic
# Reviews 2-4 relate to a second topic
# Review 2 relates to a third topic

# Old comment from gensim_tut1
# As expected, the first five documents are more strongly related to the second topic 
# while the remaining four documents to the first topic:

# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
# for doc in corpus_lsi: 
#     print(doc)

# sort documents on topic assignments
# numpy isn't the right structure for sorting based on the first column - dataframe
row = [[d[1] for d in doc] for doc in corpus_lsi]

# array = np.asarray(row).reshape(5,3)
df = pd.DataFrame(row).sort(0,axis=0,ascending=False)#0 sorts on the first column
print(df)


          0         1         2
3  0.619779  0.232931 -0.260150
1  0.549346  0.357718 -0.317442
2  0.531328  0.314275  0.698156
4  0.466477 -0.567117 -0.349622
0  0.357273 -0.681029  0.357603


In [10]:
lsi.print_topics()
# retrain model with more data
# sort documents based on topic assignments
# e.g., look at the top five documents assigned to this topic to get a sense what it's about
# you train model on many doctors' reviews, but when you do the prediction, you can project
# back onto a single doctor
# each review might contain a weightings of topics - 70%(T1)+30%(T2)
# give each new doctor a star rating per topic - that's the readout - personalized report

[u'0.197*"i" + 0.180*"patients" + 0.152*"," + 0.148*"we" + 0.135*"me" + 0.127*"dr" + 0.126*"!" + 0.116*"our" + 0.115*"do" + 0.111*"like"',
 u'-0.253*"entire" + -0.253*"amazing" + -0.253*"recommend" + -0.253*"highly" + -0.243*"dr" + -0.193*"throughout" + -0.193*"delivered" + -0.184*"we" + -0.173*"our" + -0.158*"great"',
 u'0.200*"intense" + -0.157*"we" + 0.157*"entire" + 0.157*"amazing" + 0.157*"recommend" + 0.157*"highly" + 0.146*"over" + -0.126*"our" + -0.109*"patients" + 0.101*"years"']